<a href="https://colab.research.google.com/github/kanish-dev/Music-Recommendation-System/blob/main/Music_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World!")

Hello World!


In [ ]:
from os import access
import requests
import base64

CLIENT_ID = 'XXXX'         # will be diffrent for every one create it and use
CLIENT_SECRET = 'XXXX'     # will be diffrent for every one create it and use

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type':'client_credentials'
}

response = requests.post(token_url,data=data,headers=headers)

if response.status_code ==200:
  access_token = response.json()['access_token']
  # print(access_token)
  print('Access token obtained successfully 🤩.')

else:
  print('Error in obtaining token')
  exit()


Access token obtained successfully 🤩.


In [ ]:
! pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.1/241.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id,access_token):

  # Setup spotify with access token
  sp = spotipy.Spotify(auth=access_token)

  # get the tracks from the playlist
  playlist_tracks = sp.playlist_tracks(playlist_id,fields='items(track(id,name,artists,album(id,name)))')
  # print(playlist_tracks)

  # extract relevant information and store in a list of dictonaries
  music_data = []

  for track_info in playlist_tracks['items']:
    track = track_info['track']
    track_name = track['name']
    artists = ', '.join([artist['name'] for artist in track['artists']])
    album_name = track['album']['name']
    album_id = track['album']['id']
    track_id = track['id']

    # get the audion features of the track

    audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

    # get the release date of the album

    try :
      album_info = sp.album(album_id) if album_id != 'Not available' else None
      release_date = album_info['release_date'] if album_info else None

    except:
        release_date = None

    # get the popularity of the track
    try:
      track_info = sp.track(track_id) if track_id != 'Not available' else None
      popularity = track_info['popularity'] if track_info else None

    except:
       popularity = None

     # Add additional track information to the track data
    track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,

        }
    music_data.append(track_data)
  # Create a pandas DataFrame from the list of dictionaries
  df = pd.DataFrame(music_data)
  return df

In [ ]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

music_df = get_trending_playlist_data(playlist_id, access_token)

print(music_df)

                               Track Name  \
0                         I'm Good (Blue)   
1                      Boy's a Liar Pt. 2   
2   Quevedo: Bzrp Music Sessions, Vol. 52   
3                         Me Porto Bonito   
4                             El Merengue   
..                                    ...   
95                               PUNTO 40   
96                   Marianela (Que Pasa)   
97                   Lionheart (Fearless)   
98      family ties (with Kendrick Lamar)   
99              Levitating (feat. DaBaby)   

                                     Artists  \
0                   David Guetta, Bebe Rexha   
1                  PinkPantheress, Ice Spice   
2                          Bizarrap, Quevedo   
3                Bad Bunny, Chencho Corleone   
4                  Marshmello, Manuel Turizo   
..                                       ...   
95                Rauw Alejandro, Baby Rasta   
96  HUGEL, Merk & Kremont, Lirico En La Casa   
97                   Joel C

In [ ]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


### **Now, let’s move further to building a music recommendation system**

In [ ]:
! pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df


### **While providing music recommendations to users, it is important to recommend the latest releases. For this, we need to give more weight to the latest releases in the recommendations.**

In [ ]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
  # convert the release date to datetime  object
  release_date = datetime.strptime(release_date,'%Y-%m-%d')

  # calculate the time span between the release date and today
  time_span = datetime.now() - release_date

  # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
  weight = 1/(time_span.days+1)

  return weight

In [ ]:
# Normalize the music features using Min-Max scaling

scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values

music_features_scaled = scaler.fit_transform(music_features)
print(music_features_scaled)

[[0.16025641 0.9880137  0.63636364 ... 0.44326335 0.2844135  0.56805142]
 [0.44871795 0.72089041 0.45454545 ... 0.27982992 0.87680771 0.60995148]
 [0.28846154 0.67465753 0.18181818 ... 0.25591284 0.54793787 0.56799183]
 ...
 [0.3034188  0.99143836 0.72727273 ... 0.39675791 0.33261918 0.55053205]
 [0.48076923 0.38184932 0.09090909 ... 0.25724156 0.11301553 0.61997957]
 [0.46153846 0.74828767 0.54545455 ... 0.03986181 0.93893948 0.35469482]]


### **Generate music recommendations based on the music audio features**

In [ ]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [ ]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [ ]:
input_song_name = "Call It Love"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=10)
print(f"Hybrid recommended songs for \'{input_song_name}\':")
print(recommendations)

Hybrid recommended songs for 'Call It Love':
                                   Track Name  \
7                             I'm Good (Blue)   
2                                  BABY HELLO   
8                                     Lay Low   
1  Bad Memories (feat. Elley Duhé & FAST BOY)   
0                      Mood (feat. iann dior)   
9                                       REACT   
4              Where Did You Go? (feat. MNEK)   
6                               All By Myself   
3                       Rainfall (Praise You)   
5                   Levitating (feat. DaBaby)   

                                      Artists  \
7                    David Guetta, Bebe Rexha   
2                    Rauw Alejandro, Bizarrap   
8                                      Tiësto   
1  MEDUZA, James Carter, Elley Duhé, FAST BOY   
0                         24kGoldn, iann dior   
9  Switch Disco, Ella Henderson, Robert Miles   
4                             Jax Jones, MNEK   
6                Alok, 

<ipython-input-36-3de633f34e34>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({
